<a href="https://colab.research.google.com/github/Courage-7/PandasAI-Project-For-EDA/blob/main/PandasAI_Streamlit_web_appv1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import streamlit as st
from pandasai import PandasAI
from pandasai.agent.agent import Agent
from pandasai.ee.agents.semantic_agent import SemanticAgent
from pandasai import SmartDataframe
import os

# Streamlit App Setup
st.title("Dataset Explorer with PandasAI")
st.write("Upload a dataset and start asking natural language questions to explore insights.")

# API Key Input
api_key = st.sidebar.text_input("Enter your PandasAI API Key", type="password")
if not api_key:
    st.warning("Please enter an API key to proceed.")
else:
    os.environ['PANDASAI_API_KEY'] = api_key

# Upload Dataset
uploaded_file = st.file_uploader("Upload your CSV file", type=["csv"])
if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    st.write("Dataset preview:", df.head())

    # Display sample data
    st.write(df.sample(10))

    # Initialize PandasAI with the uploaded dataframe
    sdf = SmartDataframe(df)

    # Initialize session state for storing queries and answers
    if "query_history" not in st.session_state:
        st.session_state.query_history = []

    # Query Input
    query = st.text_input("Ask your dataset question:")

    if query:
        # Use SemanticAgent for better understanding of natural queries
        agent = SemanticAgent(sdf)
        answer = agent.chat(query)

        # Store query and answer in session state
        st.session_state.query_history.append((query, answer))

        # Display the answer
        st.write("Answer:", answer)

        # Optionally show the generated code
        if st.checkbox("Show Generated Code"):
            st.code(agent.last_code_generated)

        # Display the query history
        st.write("### Previous Queries and Answers")
        for idx, (q, a) in enumerate(st.session_state.query_history):
            st.write(f"**Query {idx + 1}:** {q}")
            st.write(f"**Answer {idx + 1}:** {a}")

        # Visualization (example for fatalities query)
        if "fatalities" in query.lower():
            st.write("Rendering a plot of fatalities...")
            # Generate Plotly visualization here
            # Example: Generate a bar chart for fatalities by country (based on your data)

    # Button for downloading the report
    if st.button("Download Report"):
        # Generate a Markdown report (queries, answers, code)
        with open("report.md", "w") as file:
            file.write(f"# Report\n\n")
            for idx, (q, a) in enumerate(st.session_state.query_history):
                file.write(f"### Query {idx + 1}\n")
                file.write(f"{q}\n")
                file.write(f"### Answer {idx + 1}\n")
                file.write(f"{a}\n")
                file.write(f"### Generated Code\n")
                file.write(f"```python\n{agent.last_code_generated}\n```\n")
        st.write("Report saved as 'report.md'.")

# Dataset Reset Button
if st.button("Switch Dataset"):
    st.session_state.df = None  # Reset the dataframe
    st.write("Dataset has been reset. Please upload a new one.")
